In [1]:
import sys
import duckdb
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

from pokerkit import HandHistory
from data.utils.hand_history import load_hhs_from_db

In [2]:
con = duckdb.connect('../data/db/master.db')
id, = con.sql('select hand_id from hands where is_showdown limit 1;').fetchone()
print(id)
hh = load_hhs_from_db([id], con)[0]

3031784372


In [3]:
con.close()

In [4]:
print([s.total_pot_amount for s, _ in hh.state_actions])
print([s.status for s, _ in hh.state_actions])

[Decimal('9.0'), Decimal('9.0'), Decimal('9.0'), Decimal('9.0'), Decimal('9.0'), Decimal('9.0'), Decimal('9.0'), Decimal('30.0'), Decimal('51.0'), Decimal('69.0'), Decimal('84.0'), Decimal('84.0'), Decimal('84.0'), Decimal('84.0'), Decimal('150.0'), Decimal('216.0'), Decimal('216.0'), Decimal('216.0'), Decimal('216.0'), Decimal('390.0'), Decimal('1128.0'), Decimal('1552.50'), Decimal('1552.50'), Decimal('1552.50'), Decimal('1552.50'), Decimal('1552.50'), Decimal('0.00')]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False]


In [5]:
t = list(hh.state_actions)[-1][0]
dir(t)

['_State__high_hand_opening_lookup',
 '_State__low_hand_opening_lookup',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_begin',
 '_begin_ante_posting',
 '_begin_bet_collection',
 '_begin_betting',
 '_begin_blind_or_straddle_posting',
 '_begin_chips_pulling',
 '_begin_chips_pushing',
 '_begin_dealing',
 '_begin_hand_killing',
 '_begin_showdown',
 '_consume_cards',
 '_end',
 '_end_ante_posting',
 '_end_bet_collection',
 '_end_betting',
 '_end_blind_or_straddle_posting',
 '_end_chips_pulling',
 '_end_chips_pushing',
 '_end_dealing',
 '_end_hand_ki

In [6]:
[s.showdown_index for s, _ in hh.state_actions]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 4,
 3,
 None,
 3,
 4,
 None]

In [7]:
[s.hole_cards for s, _ in hh.state_actions]

[[[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []],
 [[], [], [], [Ks, Tc], []]]

In [8]:
[a for _, a in hh.state_actions]

[None,
 'd dh p1 ????',
 'd dh p2 ????',
 'd dh p3 ????',
 'd dh p4 ????',
 'd dh p5 ????',
 'p3 f',
 'p4 cbr 21',
 'p5 cc',
 'p1 cc',
 'p2 cc',
 'd db 4cTd2s',
 'p1 cc',
 'p2 cc',
 'p4 cbr 66',
 'p5 cc',
 'p1 f',
 'p2 f',
 'd db Kd',
 'p4 cbr 174',
 'p5 cbr 738',
 'p4 cc',
 'p5 sm ????',
 'p4 sm ????',
 'd db Kh',
 'p4 sm KsTc',
 'p5 sm 4s4d']

In [9]:
[(s.showdown_index, a) for s, a in hh.state_actions]

[(None, None),
 (None, 'd dh p1 ????'),
 (None, 'd dh p2 ????'),
 (None, 'd dh p3 ????'),
 (None, 'd dh p4 ????'),
 (None, 'd dh p5 ????'),
 (None, 'p3 f'),
 (None, 'p4 cbr 21'),
 (None, 'p5 cc'),
 (None, 'p1 cc'),
 (None, 'p2 cc'),
 (None, 'd db 4cTd2s'),
 (None, 'p1 cc'),
 (None, 'p2 cc'),
 (None, 'p4 cbr 66'),
 (None, 'p5 cc'),
 (None, 'p1 f'),
 (None, 'p2 f'),
 (None, 'd db Kd'),
 (None, 'p4 cbr 174'),
 (None, 'p5 cbr 738'),
 (4, 'p4 cc'),
 (3, 'p5 sm ????'),
 (None, 'p4 sm ????'),
 (3, 'd db Kh'),
 (4, 'p4 sm KsTc'),
 (None, 'p5 sm 4s4d')]

In [10]:
len(list(hh.state_actions))

27

In [11]:
len(list(hh.actions))

26